In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

## Medida del CRPS
def heavyside(prediction,actual):
    return prediction >= actual

def cdf_dif(prediction,actual):
    quantiles = np.arange(1,100)/100.0
    t=pd.Series(prediction)
    dif=t-t.shift(1)
    dif=dif.dropna()
    fs = sum(dif*((quantiles-heavyside(prediction,actual))[1:]**2))
    # If the actual is outside the range of the prediction, 
    # we need to account for that areas outside the range 
    if actual > prediction[-1]:
        fs += (actual-prediction[-1]) * 1
    if actual < prediction[0]:
        fs += (prediction[0]-actual) * 1
    return fs

def CRPS(predictions, actuals):
    difs_mean = [cdf_dif(predictions[i],actuals[i]) for i in range(len(actuals))]
    return np.mean(difs_mean)

def evaluate(predictions,target):

    res={}
    
    # Calculate the CRPS
    res["crps"]=CRPS(predictions,target)
    
    # Bonus useful Feature
    count = 0
    for i in range(len(target)):
        if (target[i]>predictions[i][0]) and (target[i]<predictions[i][-1]):
            count+=1
    res["count"]=count
    
    ## Calculate as well measures for the quantile 50
    total_df = pd.DataFrame(predictions)
    quantiles = np.arange(1,100)/100.0 
    total_df.columns=np.array(quantiles).astype(str)
    #RMSE       
    res["rmse"]=np.sqrt(np.mean((target-total_df["0.5"])**2))
    #MAE    
    res["mae"]=np.mean(np.abs(target-total_df["0.5"] ) )
    #Bias 
    res["bias"]=np.mean(target-total_df["0.5"])
    #Corr
    res["corr"]=np.corrcoef(target,total_df["0.5"])[0][1]

    res["all"]=predictions
    res["target"]=target
    
    return res

In [8]:
import numpy as np
import math
import pandas as pd
from tqdm import  tqdm
from sklearn.model_selection import KFold
from numpy.fft import fft
import datetime
import json
import numpy as np
from scipy import stats

def prepare_data_from_horizon(df, horizon=12):
    
    data=df.copy()

    sel = np.concatenate([[1,2,3,4],
                         [12],
                         24*np.arange(1,9),
                         12+24*np.arange(1,9)])  
    sel=np.concatenate([sel,sel-1,sel+1])  
    
    ## lagged NO2 values
    for i in sel:
        if (i>=horizon):
            data["NO2 - "+str(i)] = data["NO2"].shift(i)

    ## lagged O3 values
    for i in 24*np.arange(1,4):
        if (i>=horizon):
            data["O3 - "+str(i)] = data["O3"].shift(i)

    ## Remove empty values
    data=data.dropna()

    X=data[list(set(data.columns)-set(['DATE',"NO2","O3"]))]
    y=data["NO2"]
  
    return X, y

#"data/dataEscAgui.csv"
def get_data(path):

    df = pd.read_csv(path,sep=";")

    # Prepare data
    data = df[["DATE","SPA.NO2","SPA.O3","MACC.NO2"]].copy()
    data["DATE"]=pd.to_datetime(data["DATE"],format="%Y-%m-%d %H:%M:%S")
    data = data.sort_values("DATE")
    data.columns = ["DATE","NO2","O3","CAMS"]

    ## Remove everything from 2020
    data=data[data["DATE"].astype(str)<"2020"]

    ## Fourier Columns
    freqs = [2922,1461,209,1465,4]
    l = 35064
    n = np.arange(len(data))
    fcols = []
    for f in freqs:
        data["c"+str(f)]=np.cos(n*2*np.pi*f/l)
        fcols.append("c"+str(f))
        data["s"+str(f)]=np.cos(n*2*np.pi*f/l)
        fcols.append("s"+str(f))

    data["NO2"]=np.log1p(data["NO2"])
    data["O3"]=np.log1p(data["O3"])
    data["CAMS"]=np.log1p(data["CAMS"])

    ## Calendar Variables 
    ## Calendar Variables do not bring better results and therefore
    ## removed
    
    return data

In [9]:
import lightgbm as lgb
import numpy as np
import math
import pandas as pd
from tqdm import tqdm
from scipy import stats

class TotalLGBQuantile():
    
    def __init__(self,n_estimators,max_depth):
        self.n_estimators=n_estimators
        self.max_depth=max_depth
        self.quantiles=[0.022750131948179195,0.15865525393145707,0.5,0.8413447460685429,0.9772498680518208]
        self.estimators = []
        
    def fit(self,X_train,y_train):
        print("training !")
        for q in tqdm(self.quantiles):
            print(f"Quantile: {q}")
            reg = lgb.LGBMRegressor(n_estimators=self.n_estimators,
                                    objective= 'quantile',
                                    loss="quantile",
                                    alpha=q,
                                    random_state=2020,
                                   max_depth=self.max_depth)
                                
            reg.fit(X_train, y_train)
            self.estimators.append(reg)
        print("Done")
        
    def predict(self,X):
        predictions_gbr = []
        print("predicting")
        for reg in tqdm(self.estimators):
            predictions_gbr.append(reg.predict(X))
         
        total_pred={}
        for i in range(len(predictions_gbr)):
            total_pred[i]=predictions_gbr[i]
            
        total_df=pd.DataFrame(total_pred)

        def process_row(row):
            v = row.values
            dif_mean = np.abs(v-v[2])
            mu = v[2]
            s = np.mean([dif_mean[0]/2,dif_mean[1],dif_mean[3],dif_mean[4]/2])
            mi_norm = stats.norm(mu,s)
            quant=[]
            for quantile in np.arange(1,100)/100.0 :
                quant.append(mi_norm.ppf(quantile))
            return pd.Series(quant)
 
        total_df = total_df.apply(process_row,axis=1)
        
        return total_df.values


In [10]:
import numpy as np
import math
import pandas as pd
from tqdm import tqdm
from scipy import stats

from keras.models import Sequential
from keras.layers import Dense, LeakyReLU
from keras.callbacks import EarlyStopping
import keras.backend as K 

from sklearn.model_selection import train_test_split

def tilted_loss(q,y,f):
    e = (y-f)
    return K.mean(K.maximum(q*e, (q-1)*e), axis=-1)

class MLPQuantile():
    
    def __init__(self):

        self.estimators = []
        
    def fit(self,X_train,y_train):
        
        def MLPmodel():
            model = Sequential()
            model.add(Dense(len(X_train[0]), input_dim=len(X_train[0]), activation=LeakyReLU(alpha=0.3)))
            model.add(Dense(int(len(X_train[0])/2), activation=LeakyReLU(alpha=0.3)))
            model.add(Dense(int(len(X_train[0])/2), activation=LeakyReLU(alpha=0.3)))
            model.add(Dense(1, activation='linear'))
            return model
        
        print("training !")

        X_ttrain, X_val, y_ttrain, y_val = train_test_split(X_train,y_train,test_size=.05,random_state=2020)

        for q in [0.022750131948179195,0.15865525393145707,0.5,0.8413447460685429,0.9772498680518208]:
            print(f"Quantile: {q}")
            model = MLPmodel()
            model.compile(loss=lambda y,f: tilted_loss(q,y,f), optimizer='adadelta')
            es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=50)
            history = model.fit(X_ttrain, y_ttrain, 
                                epochs=1000, batch_size=500,  
                                verbose=1,callbacks=[es],
                                validation_data=(X_val,y_val))
            self.estimators.append(model)
        print("Done")
        
    def predict(self,X):
        predictions_gbr = []
        print("predicting")
        for reg in tqdm(self.estimators):
            predictions_gbr.append(reg.predict(X))
         
        total_pred={}
        for i in range(len(predictions_gbr)):
            total_pred[i]=predictions_gbr[i][:,0]
            
        total_df=pd.DataFrame(total_pred)

        def process_row(row):
            v = row.values
            dif_mean = np.abs(v-v[2])
            mu = v[2]
            s = np.mean([dif_mean[0]/2,dif_mean[1],dif_mean[3],dif_mean[4]/2])
            mi_norm = stats.norm(mu,s)
            quant=[]
            for quantile in np.arange(1,100)/100.0 :
                quant.append(mi_norm.ppf(quantile))
            return pd.Series(quant)
 
        total_df = total_df.apply(process_row,axis=1)
        
        return total_df.values


In [11]:
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold

In [12]:
horizon = 13

df = get_data("../2018_2019_data/28079008.csv")
X, y = prepare_data_from_horizon(df,horizon)

kf = KFold(5,shuffle=True)
train_index, test_index = list(kf.split(X))[1]

train_index = X.index.values[train_index]
test_index = X.index.values[test_index] 
        
# Filter the test index when prediction time is 10:00
ten_index = df[(df["DATE"]-timedelta(hours=horizon)).dt.hour==10].index
test_index_10 = test_index[pd.Series(test_index).isin(ten_index)]
        
# We retrieve the indexes that are related to the test indexes according to our AR model
sel = np.concatenate([[1,2,3,4],
                            [12],
                            24*np.arange(1,9),
                            12+24*np.arange(1,9)])  
sel=np.concatenate([sel,sel-1,sel+1]) 
        
all_index_related_test = set([])
for i in sel:
    all_index_related_test |= set(test_index_10+i)
        
train_index_CV = train_index[pd.Series(train_index).isin(list(all_index_related_test))]
        
X_train = X.loc[train_index_CV]
y_train = y.loc[train_index_CV]
        
X_test = X.loc[test_index_10]
y_test = y.loc[test_index_10]
        
scaler = RobustScaler()
# Fit the scaler on the training features and transform these in one go
X_train_std = scaler.fit_transform(X_train)
# Scale the test set
X_test_std = scaler.transform(X_test)
        
lin = LinearRegression()
lin.fit(X_train_std,y_train)

dif_train = y_train-lin.predict(X_train_std)
dif_test = y_test-lin.predict(X_test_std)

In [13]:
from tqdm.notebook import tqdm

In [14]:
alls=[]

for n_estimator in tqdm([500,600,700,800,900,1000,1100]):
    for max_depth in tqdm([4,6,8,10,12,14]):
        qreg = TotalLGBQuantile(n_estimators=n_estimator,max_depth=max_depth)
        
        qreg.fit(X_train,dif_train)
        pred_difs = qreg.predict(X_test)
        a=pd.DataFrame(pred_difs)
        for col in a.columns:
            a[col] += lin.predict(X_test_std) 
        
        results=evaluate((np.exp(a)-1).values,(np.exp(y_test)-1).values)
        del results["all"]
        del results["target"]
        results["n_estimator"]=n_estimator
        results["max_depth"]=max_depth
        alls.append(results)

training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting



training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting



training !


...

Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting



training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting



training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting


In [15]:
pd.DataFrame(alls).sort_values("rmse")

,crps,count,rmse,mae,bias,corr,n_estimator,max_depth
41,12.455742,448,25.665815,17.689652,4.248721,0.754538,1100,14
35,12.484751,447,25.712491,17.754079,4.239778,0.753420,1000,14
29,12.518565,445,25.760079,17.825588,4.243110,0.752379,900,14
23,12.539876,445,25.810412,17.877162,4.221367,0.751113,800,14
38,12.581603,448,25.816871,17.992562,4.511925,0.752299,1100,8
17,12.584544,447,25.884611,17.953229,4.229881,0.749359,700,14
32,12.645785,449,25.885992,18.058085,4.498206,0.750611,1000,8
11,12.604742,447,25.893047,17.947053,4.249623,0.749344,600,14
39,12.648288,446,25.938679,18.149995,4.536384,0.749023,1100,10
26,12.663594,449,25.968626,18.084489,4.548332,0.748897,900,8


In [17]:
alls=[]

for n_estimator in tqdm([1500,1700,1900]):
    for max_depth in tqdm([16,18,20]):
        qreg = TotalLGBQuantile(n_estimators=n_estimator,max_depth=max_depth)
        
        qreg.fit(X_train,dif_train)
        pred_difs = qreg.predict(X_test)
        a=pd.DataFrame(pred_difs)
        for col in a.columns:
            a[col] += lin.predict(X_test_std) 
        
        results=evaluate((np.exp(a)-1).values,(np.exp(y_test)-1).values)
        del results["all"]
        del results["target"]
        results["n_estimator"]=n_estimator
        results["max_depth"]=max_depth
        alls.append(results)
pd.DataFrame(alls).sort_values("rmse")

training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting



training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting



training !


...


training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting



training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting


,crps,count,rmse,mae,bias,corr,n_estimator,max_depth
8,12.256893,444,25.056755,17.461906,4.341796,0.769973,1900,20
5,12.257105,444,25.114507,17.501657,4.380339,0.768799,1700,20
2,12.222317,445,25.127616,17.514800,4.399304,0.768684,1500,20
6,12.249419,445,25.200286,17.389576,4.468181,0.767214,1900,16
3,12.272354,445,25.286323,17.452432,4.513827,0.765468,1700,16
0,12.273952,445,25.299871,17.483200,4.508806,0.765118,1500,16
7,12.430157,443,25.396144,17.698768,4.272616,0.761338,1900,18
4,12.464333,443,25.440041,17.727003,4.287562,0.760424,1700,18
1,12.480926,443,25.469553,17.741068,4.298643,0.759867,1500,18
